In [5]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
from urllib.request import urlretrieve

torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(linewidth = 125)
np.set_printoptions(linewidth = 125)

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

# Open the compressed file and take out our data that we want then convert all of the separate datasets into tensors
with gzip.open(path_gz, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1') 
    x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

In [19]:
# Basic architecture

# rows and columns of our data - rows for each image and columns for each pixel of each image
n, m = x_train.shape
# c is the total outputs
c = y_train.max() + 1 # possible outputs since we also have the number 0 included

# number of hidden nodes
nh = 50

# Weights and Biases
# Start with each pixel being an input - passing 1 image at a time
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
# Then we add a bias to all those hidden nodes
# Finally we have our second layer which has a weight for all of the 
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

In [32]:
def lin(x, w, b): 
    return x@w + b

t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [33]:
def relu(x):
    return x.clamp_min(0.)

In [34]:
t = relu(t)
t

tensor([[ 0.0000,  1.4440,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 3.8119,  0.0000, 10.9341,  ...,  3.7241,  0.0000,  0.0000],
        [11.3077,  0.0000,  6.6042,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [15.4602,  0.0000,  6.8516,  ...,  0.0000,  0.0000,  0.0000],
        [ 7.7826,  2.3167,  6.9597,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  8.5584,  ...,  6.1868,  0.0000,  0.0000]])

In [39]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

res = model(x_valid)
res.shape

torch.Size([10000, 1])

In [40]:
y_valid.shape

torch.Size([10000])

In [41]:

res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [44]:
(res-y_valid[:, None]).shape
# instead of taking the minus of both, we want to make sure that both follow the same rules
# otherwise it will insert the unit axis of both
# so we can add another dimension with none to make it work
# so we take all rows and add 1 column with None

# can also do res.squeeze()

torch.Size([10000, 1])

In [45]:
y_train,y_valid = y_train.float(),y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [46]:
def mse(output, target):
    return (output[:, 0] - target).pow(2).mean()

mse(preds, y_train)

tensor(1825.7338)

In [51]:
from sympy import symbols, diff

x, y = symbols('x y')
y


diff(3*x**2+9, x)

6*x

In [52]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [53]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    # When we do the forward pass we calculate the loss by doing output - target
    loss = diff.pow(2).mean()
    
    # backward pass - we contain the gradients as a property of the tensors
    # 
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [55]:
forward_and_backward(x_train, y_train)

# Save for testing against later
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

In [56]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [57]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [58]:
loss = forward(xt2, y_train)
loss.backward()

In [59]:
# Refactoring as classes

class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self):
        # multiplied due to the chain rule
        self.inp.g = (self.inp > 0).float() * self.out.g
        
class Lin():
    def __init__(self, w, b): 
        self.w, self.b = w, b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)
        
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [61]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): 
            l.backward()
            
model = Model(w1, b1, w2, b2)
model

In [62]:
loss = model(x_train, y_train)

In [63]:
model.backward()

In [64]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

In [65]:

class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [66]:
from torch import nn
import torch.nn.functional as F

# we can now import nn module from pytorch

class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        # Import from nn.Module
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
        
    def forward(self, inp): 
        return inp @ self.w + self.b
     
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in, nh), nn.ReLU(), Linear(nh, n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        return F.mse_loss(x, targ[:,None])       

In [67]:
def log_softmax(x):
    # sum over the last dimension
    # leavees the unit axis back in that state so we dont get a out of product dimension error
    return (x.exp() / (x.exp().sum(-1, keepdim = True))).log()

# is the simplified version
def log_softmax(x): 
    return x - x.exp().sum(-1,keepdim=True).log()

# larger numbers have a smaller precision - we want more precision so we want smaller numbers
# by calculating the max of x and then we take the difference between the largest and every value
# we can subtract a from all the exponents

# LogSumExp Trick - allows us to have more accurate and smaller numbers
def logsumexp(x):
    # find the maximum of the last dimension
    m = x.max(-1)[0]
    return m + (x-m[:,None]).exp().sum(-1).log()

def log_softmax(x): return x - x.logsumexp(-1,keepdim=True)

# To optimise this we can simply index the log softmax values whislt also using the logsumexp trick
# This will make it far faster allowing us to index the cross entropy loss

In [68]:
# Cross entropy - negative log likelihood loss is cross entropy loss
# F.cross_entropy

def nll(input, target): 
    return -input[range(target.shape[0]), target].mean()

# function to find the index of the highest number is called torch.argmax()

In [ ]:
# function to calculate the accuracy is

def accuracy(out, yb):
    return (out.argmax(dim = 1) == yb).float().mean()